In [38]:
# from keras.layers import Input, Lambda, Dense, Flatten
# from keras.models import Model
# from keras.applications.vgg16 import VGG16
# from keras.applications.vgg16 import preprocess_input
# from keras.preprocessing import image
# from tensorflow.keras.preprocessing.image import ImageDataGenerator
# from keras.models import Sequential
# import numpy as np
# from glob import glob
# import matplotlib.pyplot as plt

# # re-size all the images to this
# IMAGE_SIZE = [224, 224]

# train_dir = '/Users/vishnuvardhanvankayalapati/Desktop/FinalProject_Bird_Image_Classification/Project_Data_Set/train'
# test_dir = '/Users/vishnuvardhanvankayalapati/Desktop/FinalProject_Bird_Image_Classification/Project_Data_Set/test'

# # add preprocessing layer to the front of VGG
# vgg = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

# # don't train existing weights
# for layer in vgg.layers:
#   layer.trainable = False



#   # useful for getting number of classes
# folders = glob('/Users/vishnuvardhanvankayalapati/Desktop/FinalProject_Bird_Image_Classification/Project_Data_Set/train/*')


# # our layers - you can add more if you want
# x = Flatten()(vgg.output)
# # x = Dense(1000, activation='relu')(x)
# prediction = Dense(len(folders), activation='softmax')(x)

# # create a model object
# model = Model(inputs=vgg.input, outputs=prediction)

# # view the structure of the model
# # model.summary()

# # tell the model what cost and optimization method to use
# model.compile(
#   loss='categorical_crossentropy',
#   optimizer='adam',
#   metrics=['accuracy']
# )

# train_datagen = ImageDataGenerator(rescale = 1./255)

# test_datagen = ImageDataGenerator(rescale = 1./255)

# training_set = train_datagen.flow_from_directory('/Users/vishnuvardhanvankayalapati/Desktop/FinalProject_Bird_Image_Classification/Project_Data_Set/train',
#                                                  target_size = (224, 224),
#                                                  batch_size = 32,
#                                                  class_mode = 'categorical')

# test_set = test_datagen.flow_from_directory('/Users/vishnuvardhanvankayalapati/Desktop/FinalProject_Bird_Image_Classification/Project_Data_Set/test',
#                                             target_size = (224, 224),
#                                             batch_size = 32,
#                                             class_mode = 'categorical')

# # fit the model
# r = model.fit(
#   training_set,
#   validation_data=test_set,
#   epochs=10,
#   steps_per_epoch=len(training_set),
#   validation_steps=len(test_set)
# )

In [39]:
# # Evaluate the model on training data
# train_scores = model.evaluate(training_set, steps=len(training_set))
# print("Training Accuracy: %.2f%%" % (train_scores[1] * 100))

# # Evaluate the model on testing data
# test_scores = model.evaluate(test_set, steps=len(test_set))
# print("Testing Accuracy: %.2f%%" % (test_scores[1] * 100))


In [40]:
# # Extract training and validation accuracy from history
# training_accuracy = (r.history['accuracy'][-2] * 100)
# validation_accuracy = (r.history['val_accuracy'][-2] * 100)

# print(f"Training Accuracy: {training_accuracy}")
# print(f"Validation Accuracy: {validation_accuracy}")

In [41]:

# import tensorflow as tf

# from keras.models import load_model

# model.save('Vgg16_model_15.h5')

In [42]:
# import numpy as np
# from keras.preprocessing import image

# img_height = 224
# img_width = 224

# # Load the image
# img_path = '/Users/vishnuvardhanvankayalapati/Desktop/FinalProject_Bird_Image_Classification/Project_Data_Set/test/Cardinal/Cardinal_0094_17165.jpg'
# img = image.load_img(img_path, target_size=(img_height, img_width))

# # Preprocess the image
# img_array = image.img_to_array(img)
# img_array = np.expand_dims(img_array, axis=0)
# img_array /= 255.  # Normalize the image

# # Make predictions
# predictions = model.predict(img_array)
 
# # Get the predicted class
# predicted_class = np.argmax(predictions[0])

# # Define the ground truth label
# ground_truth = 3  # The ground truth label for the given image is cardinal

# # Calculate prediction accuracy
# accuracy = (predicted_class == ground_truth)

# # Print the predicted class and probability
# print("Predicted Class:", predicted_class)
# print("Predicted Probability:", predictions[0][predicted_class])

# # Print the prediction accuracy
# print("Prediction Accuracy:", accuracy)


In [43]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.models import Model
import numpy as np
import os
from keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.optimizers import Adam
from sklearn.metrics import classification_report, confusion_matrix

base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# data generator for train and test data

train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

train_dir = '/Users/vishnuvardhanvankayalapati/Desktop/FinalProject_Bird_Image_Classification/Project_Data_Set/train'
test_dir = '/Users/vishnuvardhanvankayalapati/Desktop/FinalProject_Bird_Image_Classification/Project_Data_Set/test'

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=True
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)

# Create the classifier model
model = Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(5, activation='softmax'))

# Freeze the base model's layers
base_model.trainable = False

model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])

model_vgg = model.fit(train_generator, steps_per_epoch=len(train_generator), epochs=100)

# Evaluate the model on the test set
y_pred = model.predict(test_generator)

#y_pred = (y_pred > 0.5)
y_pred = np.argmax(y_pred, axis=1)

# Get the true labels for the test set
y_true = test_generator.classes

# Calculate the confusion matrix
cm = confusion_matrix(y_true, y_pred)
print('Confusion Matrix:')
print(cm)

# Create the classification report
class_names = list(test_generator.class_indices.keys())
report = classification_report(y_true, y_pred, target_names=class_names)
print('Classification Report:')
print(report)

# model.save("/content/drive/My Drive/OCR/Stimulation/vgg16NDC.keras")

Found 200 images belonging to 5 classes.
Found 88 images belonging to 5 classes.
Epoch 1/100


/opt/anaconda3/lib/python3.11/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


7/7 ━━━━━━━━━━━━━━━━━━━━ 40s 5s/step - accuracy: 0.3279 - loss: 45.0846
Epoch 2/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 3/100


2024-03-14 13:20:09.278681: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
/opt/anaconda3/lib/python3.11/contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


7/7 ━━━━━━━━━━━━━━━━━━━━ 38s 6s/step - accuracy: 0.8795 - loss: 5.1040
Epoch 4/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 861us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 5/100


...

7/7 ━━━━━━━━━━━━━━━━━━━━ 39s 5s/step - accuracy: 1.0000 - loss: 2.4571e-08
Epoch 98/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 720us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 99/100


2024-03-14 13:51:22.734007: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


7/7 ━━━━━━━━━━━━━━━━━━━━ 39s 5s/step - accuracy: 1.0000 - loss: 2.1637e-08
Epoch 100/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 886us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00


2024-03-14 13:52:01.589795: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


3/3 ━━━━━━━━━━━━━━━━━━━━ 17s 5s/step
Confusion Matrix:
[[17  0  0  1  0]
 [ 4  7  1  3  3]
 [ 0  0 18  0  0]
 [ 0  0  0 18  0]
 [ 0  0  0  0 16]]
Classification Report:
                     precision    recall  f1-score   support

           Bobolink       0.81      0.94      0.87        18
   Brewer_Blackbird       1.00      0.39      0.56        18
           Cardinal       0.95      1.00      0.97        18
  Groove_billed_Ani       0.82      1.00      0.90        18
Red_faced_Cormorant       0.84      1.00      0.91        16

           accuracy                           0.86        88
          macro avg       0.88      0.87      0.84        88
       weighted avg       0.88      0.86      0.84        88

